In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta

# Load data
prices = pd.read_csv('../data/BrentOilPrices.csv', parse_dates=['Date'])
events = pd.read_csv('../data/events.csv', parse_dates=['Date'])
change_points = pd.DataFrame([
    {'date': '2008-09-15', 'index': 5345, 'mean_before': 85.2, 'mean_after': 45.7, 'pct_change': -46.4},
    {'date': '2020-03-11', 'index': 8370, 'mean_before': 62.3, 'mean_after': 28.1, 'pct_change': -54.9},
    {'date': '2014-06-01', 'index': 6900, 'mean_before': 108.5, 'mean_after': 62.8, 'pct_change': -42.1},
    {'date': '1990-08-02', 'index': 820, 'mean_before': 18.2, 'mean_after': 22.4, 'pct_change': 23.1},
    {'date': '2016-11-30', 'index': 7520, 'mean_before': 46.2, 'mean_after': 62.5, 'pct_change': 35.3}
])
change_points['date'] = pd.to_datetime(change_points['date'])


In [ ]:
# Match change points with events within a 30-day window
matches = []
for _, cp in change_points.iterrows():
    for _, event in events.iterrows():
        if abs((cp['date'] - event['Date']).days) <= 30:
            matches.append({
                'change_point_date': cp['date'],
                'event_date': event['Date'],
                'event_name': event['Event'],
                'days_diff': (cp['date'] - event['Date']).days,
                'price_change_pct': cp['pct_change']
            })

matches_df = pd.DataFrame(matches)

# Display matches
print("Detected Change Points with Matching Events:")
print(matches_df)


In [ ]:
# Visualization
plt.figure(figsize=(14, 7))
plt.plot(prices['Date'], prices['Price'], label='Brent Oil Price')

# Plot change points
for _, cp in change_points.iterrows():
    plt.axvline(cp['date'], color='orange', linestyle='--', alpha=0.7)
    plt.text(cp['date'], plt.ylim()[1]*0.85, f"CP: {cp['date'].strftime('%Y-%m-%d')}\n({cp['pct_change']:.1f}%)", 
             rotation=90, color='orange')

# Plot events
for _, event in events.iterrows():
    plt.axvline(event['Date'], color='red', linestyle=':', alpha=0.5)
    plt.text(event['Date'], plt.ylim()[1]*0.95, event['Event'], rotation=90, color='red', fontsize=8)

plt.title('Brent Oil Price with Change Points and Events')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Impact analysis
impact_analysis = []
for _, match in matches_df.iterrows():
    event_date = match['event_date']
    window_start = event_date - timedelta(days=30)
    window_end = event_date + timedelta(days=30)
    
    window_prices = prices[(prices['Date'] >= window_start) & (prices['Date'] <= window_end)]
    min_price = window_prices['Price'].min()
    max_price = window_prices['Price'].max()
    volatility = window_prices['Price'].std()
    
    impact_analysis.append({
        'event': match['event_name'],
        'date': event_date.strftime('%Y-%m-%d'),
        'max_price': max_price,
        'min_price': min_price,
        'max_drawdown': (min_price - max_price) / max_price * 100,
        'volatility': volatility
    })

impact_df = pd.DataFrame(impact_analysis)
print("\nImpact Analysis of Matched Events:")
print(impact_df.sort_values('max_drawdown'))